## Application #1: nbconvert


### Additional resource:

Andreas Mueller: [*Writing (and publishing) a book written in Jupyter notebooks*](https://conferences.oreilly.com/jupyter/jup-ny/public/schedule/detail/60087)

## Distinction between use cases for nbformat and nbconvert

`nbformat`: creating and validating notebooks 

`nbconvert`: manipulate existing notebooks.


## Nbconvert use cases

* converting notebooks into other formats
    * web-display: html, slides (with reveal.js)
    * publishable documents: LaTeX/PDF, ASCIIDoc
    * plain-text: rst, markdown
    * executable scripts: e.g., `*.py` 

## Nbconvert use cases (cont.)

* manipulating notebook content 
    * cell magic (`%%R`) code highlighting
    * removing content
    * extracting images references for plain-text formats (LaTeX, markdown)
* executing notebooks from the command-line

## Command Line Interface

In [1]:
!jupyter nbconvert combined_nb.ipynb --to markdown
!cat combined_nb.md


[NbConvertApp] Converting notebook ../combined_nb.ipynb to markdown
[NbConvertApp] Writing 79812 bytes to ../combined_nb.md

# The Jupyter Notebook as Document:<br>from structure to application

### M Pacer<br>Damián Avila<br> Jess Hamrick

# JupyterLab User Testing!

**What**: 5–30 mins of making JupyterLab better  
**When**: Friday from 8am to 5:30pm  
**Where**: 2nd floor, Gramercy  
**How**: Walk in or sign up at: https://bit.ly/jupytercon-usertesting

## Outline

1. [Introduction: *.ipynb and nbformat](#Introduction:-*.ipynb-and-nbformat)
2. [Application #1: nbconvert](#Application-#1:-nbconvert)
3. [Application #2: Nikola](#Application-#2:-Nikola)
4. [Application #3: RISE](#Application-#3:-RISE)
5. [Application #4: nbgrader](#Application-#4:-nbgrader)

## Introduction: `*.ipynb` and nbformat

###  `*.ipynb`

* JSON on-disk representation
* versions define json.schema
* current major version: 4

Straightforward questions: 

1. **Minimal structure** needed to meet the schema?
2. **

## NbConvertApp

* manages the CLI.

* manages the configuration as established both on the command line and via traitlet config files.

* wraps base class functionality.

### Nbconvert configuration: traitlets

Configuration is specified using traitlets
- as on-disk config file: `jupyter_nbconvert_config.(py|json)`
- command line arguments: `jupyter nbconvert --to markdown`
- passed to instance: `Exporter(config=Config())`

## Base Classes

* [Exporters](#Exporters)
* [Preprocessors](#Preprocessors)
* [Templates (and filters)](#Templates-(and-filters)
* [Writers](#Writers-&-Postprocessors)
* [Postprocessors](#Writers-&-Postprocessors)

## Exporters

Orchestration layer.

Keyed to the `--to <exporter_name>` command line argument.

Exporters specify many aspects of conversion pipeline. E.g.:
* which preprocessors 
* whether & which template
* output format

### The resources dictionary

In addition the `NotebookNode` instance, exporters create a resources dictionary. 

This is useful for passing information that not be in the notebook itself.
* Notebook styling (for html export)
* metadata for populating a jekyll header

## Preprocessors

Notebook to notebook transformations.

**ExecutePreprocessor**: This enables CLI execution.  
**TagRemovePreprocessor**: Removes cells tagged with particular tags specified as traitlets.

## Templates (and filters)

Nbconvert uses Jinja2 templates. 

Templates inherit from one another.

Templates can access `filters` that can transform the content passed through them.

One of the most common filters passes the plaintext representation of a cell's source to `pandoc` for conversion.

### pandoc & pandocfilters

`pandoc` converts between formats

`pandocfilters` manipulate pandoc's intermediate `JSON` representation.

##  Writers & Postprocessors

**Writers** handles the Exporter's final output.

`FilesWriter` writes to disk.

**Postprocessors** manipulate the file after the Writer is finished.

`ServePostProcessor` serves html file.

## Entrypoints

In addition to being highly configurable, we have a mechanism for 3rd party libraries to register `Exporters` using entrypoints. 

1. Define `JekyllExporter` 
2. package it in `my_jekyll_exporter`

```python
setup(…,
      entry_points = {
        'nbconvert.exporters': 
            ['jekyll = my_jekyll_exporter:JekyllExporter']
      }
)
```

### Entrypoints (contd.)
And with `pip install my_package`…

Anyone can use your exporter with  
`jupyter nbconvert --to jekyll` 



## Multi-notebook workflows

Nbconvert works on single notebooks. 

Multiple notebook conversion is still in early days.

Examples:
- [bookbook](https://github.com/takluyver/bookbook) (html or pdf books from multiple notebooks)
- [multi_rise](https://github.com/mpacer/multi_rise) (one slideshow from many notebooks)